In [1]:
# Load dependencies
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
 
[WDM] - Driver [C:\Users\DELGCA01\.wdm\drivers\chromedriver\win32\86.0.4240.22\chromedriver.exe] found in cache


## NASA Mars page load

In [3]:
url = 'https://mars.nasa.gov/news/'
response = requests.get(url)
soup = bs(response.text, 'html.parser')

## Scrape NASA Mars News Titles and Paragraphs

In [4]:
news_titles = soup.find('div', class_="content_title")
news_titles.text.strip()

'NASA to Broadcast Mars 2020 Perseverance Launch, Prelaunch Activities'

In [5]:
news_paragraph = soup.find('div',class_='rollover_description_inner')
news_paragraph.text.strip()

'Starting July 27, news activities will cover everything from mission engineering and science to returning samples from Mars to, of course, the launch itself.'

## Scrape JPL Mars Space Images

In [6]:
base_url = "https://www.jpl.nasa.gov"
image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(image_url)
html = browser.html
image_page = bs(html,'html.parser')

In [7]:
featured_image = image_page.find('img',class_='thumb')
featured_image_url = base_url + featured_image['src']
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA24247-640x350.jpg'

## Mars Facts scrape site

In [8]:
#Navigate to the next site to scrape
mars_facts_url = "https://space-facts.com/mars/"
mars_facts = pd.read_html(mars_facts_url)[2]
mars_facts.columns =["Data","Value"]
mars_facts


,Data,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [9]:
mars_facts_table_html = mars_facts.to_html()
print(mars_facts_table_html)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Data</th>
      <th>Value</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>7</th>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>8</th>
      <td>R

## Mars Hemisphere Scrape Site

In [10]:
base_url = "https://astrogeology.usgs.gov"
mars_hemi_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(mars_hemi_url)
html = browser.html
soup = bs(html,'html.parser')
hemispheres_list = soup.find('div', id='product-section')

In [11]:
hemispheres = []

titles = hemispheres_list.find_all("h3")
for title in titles:
    hemispheres.append(title.text)
print(hemispheres)

['Cerberus Hemisphere Enhanced', 'Schiaparelli Hemisphere Enhanced', 'Syrtis Major Hemisphere Enhanced', 'Valles Marineris Hemisphere Enhanced']


In [12]:
hemi_desc = hemispheres_list.find_all("div", class_="description")
print(hemi_desc)

[<div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p></div>, <div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><h3>Schiaparelli Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 35 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Schiaparelli hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. The images were acquired in 1980 during early northern…</p></div>, <div class="description"><a cl

In [13]:
hemisphere_image_urls = []

for i in hemi_desc:
    hemisphere_dict = {}
    
    title = i.find('h3').text
    hemisphere_dict['title']= title
    link = i.find('a')['href']
    page = base_url+link
    
    # Navigate the new page
    browser.visit(page)
    hemisphere_html = browser.html
    hemisphere_page = bs(hemisphere_html,'html.parser')
    
    # save img url
    img_url = hemisphere_page.find('li').a['href']
    hemisphere_dict['img_url']= img_url
    hemisphere_image_urls.append(hemisphere_dict)

hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [14]:
browser.quit()